In [ ]:
import os

# --- 1. DETECT LOCATION & SETUP PATHS ---
# If running inside 'notebooks', save to the parent folder "../src"
if os.getcwd().endswith("notebooks"):
    print(f"Detected execution inside '{os.getcwd()}'. Saving files to parent directory...")
    base_dir = "../src"
else:
    print(f"Detected execution in root. Saving files to 'src/' directory...")
    base_dir = "src"

# Create the directory
os.makedirs(base_dir, exist_ok=True)

# --- FILE 1: __init__.py ---
init_code = """
from .etl import load_data
from .feature_eng import engineer_features
from .predictive_model import train_skip_model
from .clustering import cluster_listeners
"""
with open(os.path.join(base_dir, "__init__.py"), "w", encoding='utf-8') as f:
    f.write(init_code)
print(f"Created: {base_dir}/__init__.py")

# --- FILE 2: etl.py ---
etl_code = """
import pandas as pd
import json
import os
import glob

def load_data(raw_data_path):
    print(f"ETL Started: Looking for data in {raw_data_path}...")
    files = glob.glob(os.path.join(raw_data_path, "*.json"))
    
    if not files:
        raise FileNotFoundError(f"No JSON files found in {raw_data_path}. Please move your files there.")
    
    data_frames = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            data_frames.append(pd.DataFrame(data))
            
    df = pd.concat(data_frames, ignore_index=True)
    
    if 'ts' in df.columns:
        df['ts'] = pd.to_datetime(df['ts'])
    
    df['master_metadata_track_name'] = df['master_metadata_track_name'].fillna('Unknown Track')
    df['master_metadata_album_artist_name'] = df['master_metadata_album_artist_name'].fillna('Unknown Artist')
    
    if 'ms_played' in df.columns and 'skipped' in df.columns:
        df = df[(df['ms_played'] > 10000) | (df['skipped'] == True)]
        
    print(f"✅ ETL Complete. Processed {len(df)} rows.")
    return df
"""
with open(os.path.join(base_dir, "etl.py"), "w", encoding='utf-8') as f:
    f.write(etl_code)
print(f"✅ Created: {base_dir}/etl.py")

# --- FILE 3: feature_eng.py ---
feat_code = """
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def engineer_features(df):
    print("⚙️ Feature Engineering Started...")
    df = df.copy()
    
    df['hour'] = df['ts'].dt.hour
    df['day_of_week'] = df['ts'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    if 'skipped' in df.columns:
        df['is_skipped'] = df['skipped'].astype(int)
    else:
        df['is_skipped'] = 0
        
    if 'reason_start' in df.columns:
        le = LabelEncoder()
        df['reason_start_encoded'] = le.fit_transform(df['reason_start'].astype(str))
        
    if 'shuffle' in df.columns:
        df['shuffle_feature'] = df['shuffle'].astype(int)
        
    return df
"""
with open(os.path.join(base_dir, "feature_eng.py"), "w", encoding='utf-8') as f:
    f.write(feat_code)
print(f"Created: {base_dir}/feature_eng.py")

# --- FILE 4: predictive_model.py ---
model_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def train_skip_model(df):
    print("Training Predictive Model (Random Forest)...")
    
    features = ['ms_played', 'hour', 'day_of_week', 'is_weekend']
    if 'reason_start_encoded' in df.columns:
        features.append('reason_start_encoded')
    if 'shuffle_feature' in df.columns:
        features.append('shuffle_feature')
        
    X = df[features]
    y = df['is_skipped']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf_model.fit(X_train, y_train)
    
    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Model Results - AUC Score: {auc_score:.2f}")
    return rf_model, auc_score, features
"""
with open(os.path.join(base_dir, "predictive_model.py"), "w", encoding='utf-8') as f:
    f.write(model_code)
print(f"Created: {base_dir}/predictive_model.py")

# --- FILE 5: clustering.py ---
cluster_code = """
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def cluster_listeners(df):
    print("Running K-Means Clustering on Artists...")
    
    artist_stats = df.groupby('master_metadata_album_artist_name').agg({
        'ms_played': 'sum',
        'is_skipped': 'mean',
        'ts': 'count'
    }).reset_index()
    
    artist_stats.columns = ['Artist', 'Total_Ms', 'Skip_Rate', 'Play_Count']
    data = artist_stats[artist_stats['Play_Count'] > 20].copy()
    
    scaler = StandardScaler()
    X = scaler.fit_transform(data[['Skip_Rate', 'Play_Count']])
    
    kmeans = KMeans(n_clusters=3, random_state=42)
    data['Cluster'] = kmeans.fit_predict(X)
    
    print("Clustering Complete.")
    return data
"""
with open(os.path.join(base_dir, "clustering.py"), "w", encoding='utf-8') as f:
    f.write(cluster_code)
print(f"Created: {base_dir}/clustering.py")

print("\n SUCCESS! All Python files have been created.")

In [ ]:
import pandas as pd
import json
import os
import glob

# 1. Find the file
raw_path = os.path.join('..', 'data', 'raw')
files = glob.glob(os.path.join(raw_path, "*.json"))

if files:
    print(f"found file: {files[0]}")
    # 2. Load just the first record to check columns
    with open(files[0], 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    df_preview = pd.DataFrame(data)
    print("\n--- YOUR COLUMNS ARE: ---")
    print(list(df_preview.columns))
else:
    print("No files found!")

In [ ]:
import os

# --- DETERMINE PATH ---
# If running inside 'notebooks', save to '../src'
if os.getcwd().endswith("notebooks"):
    base_dir = "../src"
else:
    base_dir = "src"

# --- 1. UPDATE ETL (Handle Column Names) ---
etl_code = """
import pandas as pd
import json
import os
import glob

def load_data(raw_data_path):
    print(f"ETL Started: Looking for data in {raw_data_path}...")
    files = glob.glob(os.path.join(raw_data_path, "*.json"))
    
    if not files:
        raise FileNotFoundError(f"No JSON files found in {raw_data_path}.")
    
    data_frames = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            data_frames.append(pd.DataFrame(data))
            
    df = pd.concat(data_frames, ignore_index=True)
    
    # --- FIX FOR STANDARD DATA FORMAT ---
    # Rename your columns to match the project standard
    rename_map = {
        "endTime": "ts",
        "artistName": "master_metadata_album_artist_name",
        "trackName": "master_metadata_track_name",
        "msPlayed": "ms_played"
    }
    df = df.rename(columns=rename_map)
    
    # Basic Formatting
    df['ts'] = pd.to_datetime(df['ts'])
    df['master_metadata_track_name'] = df['master_metadata_track_name'].fillna('Unknown Track')
    df['master_metadata_album_artist_name'] = df['master_metadata_album_artist_name'].fillna('Unknown Artist')
    
    print(f"ETL Complete. Processed {len(df)} rows.")
    return df
"""
with open(os.path.join(base_dir, "etl.py"), "w", encoding='utf-8') as f:
    f.write(etl_code)
print("Updated: src/etl.py (Compatible with your data)")

# --- 2. UPDATE FEATURE ENGINEERING (Create 'Skip' Logic) ---
feat_code = """
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def engineer_features(df):
    print("Feature Engineering Started...")
    df = df.copy()
    
    # Time Features
    df['hour'] = df['ts'].dt.hour
    df['day_of_week'] = df['ts'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    # --- PROXY LABELING FOR RESUME ---
    # Since 'skipped' column is missing, we assume:
    # Play time < 30 seconds (30000ms) = SKIPPED
    if 'skipped' not in df.columns:
        df['is_skipped'] = df['ms_played'].apply(lambda x: 1 if x < 30000 else 0)
    else:
        df['is_skipped'] = df['skipped'].astype(int)
        
    # Standardize other columns if missing
    if 'reason_start' in df.columns:
        le = LabelEncoder()
        df['reason_start_encoded'] = le.fit_transform(df['reason_start'].astype(str))
        
    return df
"""
with open(os.path.join(base_dir, "feature_eng.py"), "w", encoding='utf-8') as f:
    f.write(feat_code)
print("Updated: src/feature_eng.py (Added Skip Logic)")

In [ ]:
import sys
import os
import importlib # <--- ADDED: To force reload of fixed modules
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Setup Path
sys.path.append(os.path.abspath('..'))

# 2. Import Modules
import src.etl
import src.feature_eng
import src.predictive_model
import src.clustering

# 3. FORCE RELOAD (Critical: Ensures the "Fix" you just made is actually used)
importlib.reload(src.etl)
importlib.reload(src.feature_eng)
importlib.reload(src.predictive_model)
importlib.reload(src.clustering)

# 4. Import functions from reloaded modules
from src.etl import load_data
from src.feature_eng import engineer_features
from src.predictive_model import train_skip_model
from src.clustering import cluster_listeners

# --- EXECUTION ---

try:
    print("Starting Pipeline...")

    # A. Load Data
    raw_path = os.path.join('..', 'data', 'raw') 
    df = load_data(raw_path)
    
    # DEBUG: Check if column renaming worked
    print(f"Columns after ETL: {list(df.columns)[:5]}...")
    
    # B. Feature Engineering
    df_features = engineer_features(df)
    print(f"   ⚙️ Features Created. Skip Rate calculated: {df_features['is_skipped'].mean():.2%}")
    
    # C. Run Predictive Model (Resume Validation)
    print("\n-----------------------------------")
    rf_model, auc, features = train_skip_model(df_features)
    print(f"RESUME PROOF: Model AUC Score is {auc:.2f}")
    print("-----------------------------------\n")

    # D. Run Clustering (Resume Validation)
    clustered_df = cluster_listeners(df_features)
    
    # --- VISUALIZATION ---
    
    # Graph 1: Feature Importance
    plt.figure(figsize=(10, 5))
    importances = rf_model.feature_importances_
    indices = range(len(importances))
    plt.barh(indices, importances, align='center')
    plt.yticks(indices, features)
    plt.xlabel('Relative Importance')
    plt.title('Why do I skip songs? (Feature Importance)')
    plt.grid(True, alpha=0.3)
    plt.show()

    # Graph 2: Clusters
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=clustered_df, x='Play_Count', y='Skip_Rate', hue='Cluster', palette='viridis', s=100)
    plt.title('My Music Taste Clusters')
    plt.xlabel('Play Count')
    plt.ylabel('Skip Rate')
    plt.grid(True, alpha=0.3)
    plt.show()

except FileNotFoundError as e:
    print(f"\n ERROR: {e}")
    print("Check 'data/raw' folder!")
except KeyError as e:
    print(f"\n KEY ERROR: {e}")
    print("The ETL fix didn't apply. Please RESTART YOUR KERNEL (Kernel -> Restart) and try again.")
except Exception as e:
    print(f"\n Something went wrong: {e}")

In [ ]:
import sys
import os
import importlib

# 1. Setup Paths
sys.path.append(os.path.abspath('..'))

# 2. Import & FORCE RELOAD the Agent (Crucial step!)
import agent.chat_bot
importlib.reload(agent.chat_bot) 

from src.etl import load_data
from src.feature_eng import engineer_features
from agent.chat_bot import SpotifyAgent

# 3. Load Data & Initialize
# (We load data again to make sure everything is fresh)
raw_path = os.path.join('..', 'data', 'raw')
df = load_data(raw_path)
df = engineer_features(df)

# Initialize the NEW Agent
my_agent = SpotifyAgent(df)

# 4. RUN INTERACTIVE CHAT
print("\n" + "="*60)
print("🤖 ADVANCED SPOTIFY AGENT IS LIVE!")
print("Try asking specific questions like:")
print("  👉 'What is Time listened in mins this (month, year)?'")
print("  👉 'Who is my top artist in (year)?'")
print("  👉 'Song count top 5 songs in Oct (year)'")
print("  👉 'When do I listen on (Day), (Month) (Year)?'") 
print("  👉 'Top artist on 15th August 2024 (Specific Day)'")
print("(Type 'exit' to stop)")
print("="*60 + "\n")

while True:
    user_input = input("👤 YOU: ")
    
    if user_input.lower() in ['exit', 'quit', 'stop']:
        print("👋 Bye!")
        break
        
    try:
        response = my_agent.chat(user_input)
        print(f"🤖 AGENT: {response}\n")
    except Exception as e:
        print(f"❌ Error: {e}\n")

📂 ETL Started: Looking for data in ..\data\raw...
✅ ETL Complete. Processed 15222 rows.
⚙️ Feature Engineering Started...
🤖 Spotify Data Agent Online.
📅 Data Coverage: **May 2024** to **May 2025**
Ready for specific dates (e.g., 'Nov 24 2023' or 'Saturdays in 2024').

🤖 ADVANCED SPOTIFY AGENT IS LIVE!
Try asking specific questions like:
  👉 'What is Time listened in mins this (month, year)?'
  👉 'Who is my top artist in (year)?'
  👉 'Song count top 5 songs in Oct (year)'
  👉 'When do I listen on (Day), (Month) (Year)?'
  👉 'Top artist on 15th August 2024 (Specific Day)'
(Type 'exit' to stop)



👤 YOU:  When do I listen on (mon), (june) (2024)?


🤖 AGENT: 🕰️ **Peak Listening Time (Mons June 2024):** Around **1 PM**.



👤 YOU:  When do I listen on (sat), (aug) (2024)


🤖 AGENT: 🕰️ **Peak Listening Time (Sats August 2024):** Around **9 AM**.



👤 YOU:  When do I listen on (sat), (sep) (2024)


🤖 AGENT: 🕰️ **Peak Listening Time (Sats September 2024):** Around **1 PM**.



👤 YOU:  When do I listen on (sunday), (september) (2024)


🤖 AGENT: 🕰️ **Peak Listening Time (Sundays September 2024):** Around **12 PM**.

